In [2]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor
# Initialize parameters
L = 10  # Lattice size
N = L * L * L  # Total number of spins
J = 1.0  # Coupling constant
T = 0.01  # Temperature
n_steps = int(1e4)  # Number of Monte Carlo steps
n_threads = 10  # Number of threads

config_list = []  # List to store configurations

# Initialize 3D lattice with random spins
lattice = np.random.choice([-1, 1], size=(L, L, L))

config_list.append(lattice.copy())

# Function to compute energy of a configuration
def compute_energy(lattice):
    E = 0
    for i in range(L):
        for j in range(L):
            for k in range(L):
                nn_sum = (
                    lattice[(i+1)%L, j, k] + lattice[(i-1)%L, j, k] +
                    lattice[i, (j+1)%L, k] + lattice[i, (j-1)%L, k] +
                    lattice[i, j, (k+1)%L] + lattice[i, j, (k-1)%L]
                )
                E += -J * lattice[i, j, k] * nn_sum
    return E / 2  # Each pair counted twice

# Monte Carlo simulation for each thread
def monte_carlo_thread(thread_id, steps):
    E_sum_thread = 0
    for _ in range(steps):
        # Randomly select a spin
        i, j, k = np.random.randint(0, L, 3)
        
        # Compute energy change if this spin is flipped
        nn_sum = (
            lattice[(i+1)%L, j, k] + lattice[(i-1)%L, j, k] +
            lattice[i, (j+1)%L, k] + lattice[i, (j-1)%L, k] +
            lattice[i, j, (k+1)%L] + lattice[i, j, (k-1)%L]
        )
        dE = 2 * J * lattice[i, j, k] * nn_sum
        
        # Metropolis-Hastings algorithm
        if dE < 0 or np.random.rand() < np.exp(-dE / T):
            lattice[i, j, k] *= -1
        
        # Add the energy of the current configuration to E_sum_thread
        E_current = compute_energy(lattice)
        E_sum_thread += E_current
        config_list.append(lattice.copy())

    return E_sum_thread

# Parallel Monte Carlo simulation
E_sum = 0
with ThreadPoolExecutor(max_workers=n_threads) as executor:
    steps_per_thread = n_steps // n_threads
    futures = [executor.submit(monte_carlo_thread, i, steps_per_thread) for i in range(n_threads)]
    for future in futures:
        E_sum += future.result()

# Compute energy expectation value
E_avg = E_sum / n_steps
E_avg_per_site = E_avg / N

print(f"Energy Expectation Value: {E_avg}")
print(f"Energy Expectation Value per site: {E_avg_per_site}")


KeyboardInterrupt: 

In [53]:
def magnetization(lattice):
    return np.sum(lattice) / N

total_mag = 0
for config in config_list:
    total_mag += magnetization(config)

print(f"Magnetization Expectation Value: {total_mag / len(config_list)}")

Magnetization Expectation Value: 0.2504223577642239


In [7]:
import numpy as np
a = np.array((0,1))
b = np.array((1,0))

np.linalg.norm(a-b)

1.4142135623730951